In [52]:
import yaml
import pyodbc
from market_growth_analysis.etl.stagging import *
import pandas as pd
import numpy as np
from datetime import datetime
# Load the YAML file
with open('../../conf/global.yml', 'r') as f:
    columns = yaml.safe_load(f)

financial = pd.read_csv("../../data/processed_03/financial-prices-ratios.csv", index_col=0)

# Clean Ratios

In [53]:
columns_ratios = columns['columns_ratios']
columns_ratios_all = columns['column_id'] + columns['columns_ratios']
ratios_df = financial[columns_ratios_all]

In [54]:
for column in columns_ratios:
    alpha = 0.1
    top_percentile = 1 - (alpha/2)
    low_percentile = alpha/2

    # Calculate the alpha percentile value
    top_percentile_value = ratios_df[column].quantile(top_percentile)
    low_percentile_value = ratios_df[column].quantile(low_percentile)

    # Replace outliers with percentiles value
    ratios_df[column] = ratios_df[column].where(ratios_df[column] <= top_percentile_value, top_percentile_value)
    ratios_df[column] = ratios_df[column].where(ratios_df[column] >= low_percentile_value, low_percentile_value)

C:\Users\germa\AppData\Local\Temp/ipykernel_21756/2285555845.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratios_df[column] = ratios_df[column].where(ratios_df[column] <= top_percentile_value, top_percentile_value)
C:\Users\germa\AppData\Local\Temp/ipykernel_21756/2285555845.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratios_df[column] = ratios_df[column].where(ratios_df[column] >= low_percentile_value, low_percentile_value)


In [55]:
# # Replace -inf and inf with None
# ratios_df = ratios_df.replace([-np.inf, np.inf], None)

# Make sure types are correct
ratios_df[columns['columns_ratios']] = ratios_df[columns['columns_ratios']].astype(float)
ratios_df['PK'] = ratios_df['PK'].astype(int)
ratios_df['Date'] = pd.to_datetime(ratios_df['Date'])



C:\Users\germa\AppData\Local\Temp/ipykernel_21756/3579048468.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratios_df[columns['columns_ratios']] = ratios_df[columns['columns_ratios']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_21756/3579048468.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratios_df['PK'] = ratios_df['PK'].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_21756/3579048468.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [56]:
ratios_df.describe()

,PK,Enterprise Value (EV),EV / Revenue,EV / EBITDA,EV / EBIT,EV / Invested Capital,Free Cash Flow (FCF),EV / Free Cash Flow,EV / FCF,P/E,P/S,P/CF
count,56441.000000,56441.000000,56441.000000,56441.000000,56441.000000,56441.000000,56441.000000,56441.000000,56441.000000,56441.000000,56441.000000,56441.000000
mean,28220.000000,21613.052114,11.521184,18.387361,25.685971,524.149527,200.172297,25.973325,25.973325,19.241205,8.206646,12.226018
std,16293.257609,32612.905176,11.982016,25.897525,39.842903,343.937960,723.439077,64.931012,64.931012,33.538844,10.253405,21.029652
min,0.000000,17.563700,0.479462,-46.210182,-71.291755,1.167332,-984.582550,-114.125172,-114.125172,-48.326228,0.223285,-36.024735
25%,14110.000000,450.469435,1.685104,5.446530,-0.297887,47.971406,-54.658400,-9.389816,-9.389816,-1.558609,0.950075,1.752952
50%,28220.000000,2954.837549,4.693833,13.601307,18.367813,770.051909,4.099100,12.326092,12.326092,12.710451,2.479162,8.726359
75%,42330.000000,27058.658711,25.127316,52.209802,77.751451,770.051909,191.370000,65.217949,65.217949,33.870140,14.590719,23.056171
max,56440.000000,84205.731120,30.411756,52.209802,77.751451,770.051909,2139.582000,127.303290,127.303290,76.033985,25.975786,45.345399


In [57]:
ratios_df.head(2)

,PK,ticker,Date,Enterprise Value (EV),EV / Revenue,EV / EBITDA,EV / EBIT,EV / Invested Capital,Free Cash Flow (FCF),EV / Free Cash Flow,EV / FCF,P/E,P/S,P/CF
0,0,A,2009-10-31,8148.060066,1.818358,38.985933,77.751451,49.988099,394.0,20.680355,20.680355,-48.326228,1.237014,13.585932
1,1,A,2010-10-31,10215.772804,1.876520,13.301788,18.049069,71.942062,-456.0,-22.403011,-22.403011,11.615859,1.461200,11.079071


# Clean Income Statement

In [58]:
columns_income_statement = columns['column_id'] + columns['columns_income_statement']
income_statememnt_df = financial[columns_income_statement]

In [59]:
income_statememnt_df.describe()

,PK,Revenue,Cost Of Goods Sold,Gross Profit,Research And Development Expenses,SG&A Expenses,Other Operating Income Or Expenses,Operating Expenses,Operating Income,Total Non-Operating Income/Expense,...,Other Income,Income From Continuous Operations,Income From Discontinued Operations,Net Income,EBITDA,EBIT,Basic Shares Outstanding,Shares Outstanding,Basic EPS,EPS - Earnings Per Share
count,56441.000000,53573.000000,48448.000000,53606.000000,19840.000000,54137.000000,24815.000000,48445.000000,56250.000000,49876.000000,...,1518.000000,56327.00000,10143.000000,56327.000000,50176.000000,50173.000000,56213.000000,56183.000000,5.622300e+04,5.644100e+04
mean,28220.000000,6029.864026,3961.363393,2445.953296,332.435771,1174.369353,-358.868664,1755.042444,722.664247,-96.166983,...,15.017554,484.58942,58.520145,468.121826,1055.127824,680.721953,278.384713,283.430083,-4.213436e+03,-4.197875e+03
std,16293.257609,21782.097137,16572.646587,8284.306784,1598.571693,4643.813907,2703.842537,6059.372062,3119.518656,888.793314,...,182.201979,2333.76275,969.416018,2327.364455,4062.769915,3039.055932,1963.205567,1988.539714,6.876486e+05,6.863192e+05
min,0.000000,-8219.000000,-13697.000000,-41649.440000,-18.936000,-6231.506000,-124834.000000,-43812.160000,-30363.000000,-59155.360000,...,-371.410000,-23251.00000,-8368.000000,-22802.000000,-24504.400000,-30363.000000,-2278.226000,-2278.226000,-1.595500e+08,-1.595500e+08
25%,14110.000000,111.299000,45.357325,55.512500,6.429775,25.000000,-75.846000,56.932000,-2.559150,-60.702000,...,0.000000,-6.47700,-2.000000,-7.370500,0.331000,-5.005900,16.131000,16.383000,-3.622500e-01,-3.600000e-01
50%,28220.000000,657.100000,362.086000,283.167000,29.902000,112.419000,-6.797000,228.136000,51.667000,-5.720500,...,0.000000,28.54100,0.000000,26.957000,99.201000,52.383100,47.369000,48.297000,8.100000e-01,7.900000e-01
75%,42330.000000,3318.000000,2048.092750,1264.821000,107.288000,529.450000,0.000000,902.534100,343.000000,0.140000,...,0.000000,218.50000,10.803500,211.175000,571.249950,354.331100,148.219000,151.065500,2.330000e+00,2.280000e+00
max,56440.000000,611289.000000,463721.000000,225152.000000,73213.000000,138428.000000,43812.160000,212904.000000,119437.000000,31744.110000,...,4044.055000,99803.00000,76486.910000,99803.000000,130541.000000,119437.000000,373456.000000,373456.000000,2.089355e+05,1.771514e+05


In [60]:
# Make sure types are correct
income_statememnt_df[columns['columns_income_statement']] = income_statememnt_df[columns['columns_income_statement']].astype(float)
income_statememnt_df['PK'] = income_statememnt_df['PK'].astype(int)
income_statememnt_df['Date'] = pd.to_datetime(income_statememnt_df['Date'])

C:\Users\germa\AppData\Local\Temp/ipykernel_21756/800232268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income_statememnt_df[columns['columns_income_statement']] = income_statememnt_df[columns['columns_income_statement']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_21756/800232268.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income_statememnt_df['PK'] = income_statememnt_df['PK'].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_21756/800232268.py:4: SettingWithCopyWarning

In [61]:
income_statememnt_df.describe()

,PK,Revenue,Cost Of Goods Sold,Gross Profit,Research And Development Expenses,SG&A Expenses,Other Operating Income Or Expenses,Operating Expenses,Operating Income,Total Non-Operating Income/Expense,...,Other Income,Income From Continuous Operations,Income From Discontinued Operations,Net Income,EBITDA,EBIT,Basic Shares Outstanding,Shares Outstanding,Basic EPS,EPS - Earnings Per Share
count,56441.000000,53573.000000,48448.000000,53606.000000,19840.000000,54137.000000,24815.000000,48445.000000,56250.000000,49876.000000,...,1518.000000,56327.00000,10143.000000,56327.000000,50176.000000,50173.000000,56213.000000,56183.000000,5.622300e+04,5.644100e+04
mean,28220.000000,6029.864026,3961.363393,2445.953296,332.435771,1174.369353,-358.868664,1755.042444,722.664247,-96.166983,...,15.017554,484.58942,58.520145,468.121826,1055.127824,680.721953,278.384713,283.430083,-4.213436e+03,-4.197875e+03
std,16293.257609,21782.097137,16572.646587,8284.306784,1598.571693,4643.813907,2703.842537,6059.372062,3119.518656,888.793314,...,182.201979,2333.76275,969.416018,2327.364455,4062.769915,3039.055932,1963.205567,1988.539714,6.876486e+05,6.863192e+05
min,0.000000,-8219.000000,-13697.000000,-41649.440000,-18.936000,-6231.506000,-124834.000000,-43812.160000,-30363.000000,-59155.360000,...,-371.410000,-23251.00000,-8368.000000,-22802.000000,-24504.400000,-30363.000000,-2278.226000,-2278.226000,-1.595500e+08,-1.595500e+08
25%,14110.000000,111.299000,45.357325,55.512500,6.429775,25.000000,-75.846000,56.932000,-2.559150,-60.702000,...,0.000000,-6.47700,-2.000000,-7.370500,0.331000,-5.005900,16.131000,16.383000,-3.622500e-01,-3.600000e-01
50%,28220.000000,657.100000,362.086000,283.167000,29.902000,112.419000,-6.797000,228.136000,51.667000,-5.720500,...,0.000000,28.54100,0.000000,26.957000,99.201000,52.383100,47.369000,48.297000,8.100000e-01,7.900000e-01
75%,42330.000000,3318.000000,2048.092750,1264.821000,107.288000,529.450000,0.000000,902.534100,343.000000,0.140000,...,0.000000,218.50000,10.803500,211.175000,571.249950,354.331100,148.219000,151.065500,2.330000e+00,2.280000e+00
max,56440.000000,611289.000000,463721.000000,225152.000000,73213.000000,138428.000000,43812.160000,212904.000000,119437.000000,31744.110000,...,4044.055000,99803.00000,76486.910000,99803.000000,130541.000000,119437.000000,373456.000000,373456.000000,2.089355e+05,1.771514e+05


# Clean balance sheet

In [62]:
columns_balance_sheet = columns['column_id'] + columns['columns_balance_sheet']
balance_sheet_df = financial[columns_balance_sheet]

In [63]:
# Make sure types are correct
balance_sheet_df[columns['columns_balance_sheet']] = balance_sheet_df[columns['columns_balance_sheet']].astype(float)
balance_sheet_df['PK'] = balance_sheet_df['PK'].astype(int)
balance_sheet_df['Date'] = pd.to_datetime(balance_sheet_df['Date'])

C:\Users\germa\AppData\Local\Temp/ipykernel_21756/19496767.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_df[columns['columns_balance_sheet']] = balance_sheet_df[columns['columns_balance_sheet']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_21756/19496767.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_sheet_df['PK'] = balance_sheet_df['PK'].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_21756/19496767.py:4: SettingWithCopyWarning: 
A value is trying to b

# Clean columns_cash_flow_statement

In [64]:
columns_cash_flow_statement = columns['column_id'] + columns['columns_cash_flow_statement']
cash_flow_statement_df = financial[columns_cash_flow_statement]

In [65]:
# Make sure types are correct
cash_flow_statement_df[columns['columns_cash_flow_statement']] = cash_flow_statement_df[columns['columns_cash_flow_statement']].astype(float)
cash_flow_statement_df['PK'] = cash_flow_statement_df['PK'].astype(int)
cash_flow_statement_df['Date'] = pd.to_datetime(cash_flow_statement_df['Date'])

C:\Users\germa\AppData\Local\Temp/ipykernel_21756/475317979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cash_flow_statement_df[columns['columns_cash_flow_statement']] = cash_flow_statement_df[columns['columns_cash_flow_statement']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_21756/475317979.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cash_flow_statement_df['PK'] = cash_flow_statement_df['PK'].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_21756/475317979.py:4: SettingW

# Clean prices

In [66]:
columns_prices = columns['column_id'] + columns['columns_prices']
prices_df = financial[columns_prices]

In [67]:
prices_df.describe()

,PK,longevity,Close,Volume,Growth -1,Growth +1,Growth +5,avgGrowth -10,avgGrowth -5
count,56441.000000,56441.000000,4.865900e+04,4.865900e+04,38997.000000,36546.000000,22868.000000,19954.000000,27327.000000
mean,28220.000000,21.233111,6.776836e+07,3.125118e+07,0.176132,0.151388,0.847110,0.203652,0.188021
std,16293.257609,18.208410,8.629914e+09,1.610544e+08,1.787381,1.620397,3.606645,0.446234,0.757350
min,0.000000,0.000000,-1.205572e+00,0.000000e+00,-0.999999,-0.999999,-28.594223,-0.658006,-0.856566
25%,14110.000000,6.000000,8.740000e+00,1.092490e+06,-0.229299,-0.233530,-0.234540,0.074860,0.013485
50%,28220.000000,18.000000,1.960000e+01,5.764100e+06,0.043970,0.038462,0.361355,0.149160,0.125358
75%,42330.000000,31.000000,4.320000e+01,2.131645e+07,0.331499,0.317356,1.133922,0.248257,0.258312
max,56440.000000,120.000000,1.373400e+12,1.197745e+10,198.999997,198.999997,280.645588,11.436645,56.027803


In [68]:
# Make sure types are correct
prices_df[columns['columns_prices']] = prices_df[columns['columns_prices']].astype(float)
prices_df[['PK','longevity']] = prices_df[['PK','longevity']].astype(int)
prices_df['Date'] = pd.to_datetime(prices_df['Date'])

C:\Users\germa\AppData\Local\Temp/ipykernel_21756/3747507719.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df[columns['columns_prices']] = prices_df[columns['columns_prices']].astype(float)
C:\Users\germa\AppData\Local\Temp/ipykernel_21756/3747507719.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df[['PK','longevity']] = prices_df[['PK','longevity']].astype(int)
C:\Users\germa\AppData\Local\Temp/ipykernel_21756/3747507719.py:4: SettingWithCopyWarning: 
A value is trying to be set on

# Create Dim table

In [69]:
columns_dim = columns['columns_dim']
dim_df = financial[columns_dim]

In [70]:
dim_grouped_df = dim_df.groupby('ticker').first().reset_index()

# Save dataframes to load

In [71]:
ratios_df.to_csv("../../data/load_04/ratios_df.csv")
income_statememnt_df.to_csv("../../data/load_04/income_statememnt_df.csv")
balance_sheet_df.to_csv("../../data/load_04/balance_sheet_df.csv")
cash_flow_statement_df.to_csv("../../data/load_04/cash_flow_statement_df.csv")
prices_df.to_csv("../../data/load_04/prices_df.csv")
dim_grouped_df.to_csv("../../data/load_04/dim_grouped_df.csv")